# Ensemble Learning mit dem Titanic-Datensatz

- Der originale Datensatz kann hier abgerufen werden: https://www.kaggle.com/c/titanic
- Der hier verwendete Datensatz wurde zwecks Einfachheit bereits überarbeitet und angereichtert

## Lade Libraries und Daten

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re

from sklearn.model_selection import train_test_split

pd.options.mode.chained_assignment = None

DF = pd.read_csv('data/titanic_enh.csv')

In [2]:
DF

PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name  Sex  Age  SibSp  \
0                              Braund, Mr. Owen Harris    1    1      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...    0    2      1   
2                               Heikkinen, Miss. Laina    0    1      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)    0    2      1   
4                             Allen, Mr. William Henry    1    2      0   
..                                                 ...  ...  ...    ...   
886                              Montvila, Rev. Juozas    1    1      0   
887                       Graham, Miss. Margaret Edith    0    1      0   
888           Johnston, Miss. Catherine Helen "Carrie"    0    1      1   
889                              Behr, Mr. Karl Howell    1    1      0   
890                                Dooley, Mr. Patrick    1    1      0   

     Parch            Ticket  Fare Cabin  Embarked  Has_Cabin  FamilySize  \
0        0         A/5 21171     0   NaN         0          0           2   
1        0          PC 17599     3   C85         1          1           2   
2        0  STON/O2. 3101282     1   NaN         0          0           1   
3        0            113803     3  C123         0          1           2   
4        0            373450     1   NaN         0          0           1   
..     ...               ...   ...   ...       ...        ...         ...   
886      0            211536     1   NaN         0          0           1   
887      0            112053     2   B42         0          1           1   
888      2        W./C. 6607     2   NaN         0          0           4   
889      0            111369     2  C148         1          1           1   
890      0            370376     0   NaN         2          0           1   

     IsAlone  Title  
0          0      1  
1          0      3  
2          1      4  
3          0      3  
4          1      1  
..       ...    ...  
886        1      5  
887        1      4  
888        0      4  
889        1      1  
890        1      1  

[891 rows x 16 columns]

### Lade nötige Library/Libraries

In diesem Fall verwenden wir Decision Trees als Ausgangslage für das Ensemble Learning

In [3]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

### Trainiere drei weak learners parallel auf einem zufälligen Subset der Trainingsdaten (Bagging)

In [21]:
# Sample for KFold
df = DF.copy()
train, test = train_test_split(df, test_size=0.2)



# y1 = []
# y2 = []
# y3 = []
# Create Input- and Output-Structure and train Decision Tree Classifer
training_columns = ['Pclass', 'Sex', 'Age','Fare','Embarked', 'Has_Cabin', 'FamilySize', 'IsAlone', 'Title']

# Train three Decision Trees on subset of training data
## TODO
## Tipp: Use again the train_test_split function to get a subset of the training data for each weak learner


train_subset, test_subset = train_test_split(train, test_size=0.2)
X1 = train_subset[training_columns]
y1 = train_subset[['Survived']] 

train_subset, test_subset = train_test_split(train, test_size=0.2)
X2 = train_subset[training_columns]
y2 = train_subset[['Survived']]

train_subset, test_subset = train_test_split(train, test_size=0.2)
X3 = train_subset[training_columns]
y3 = train_subset[['Survived']]

X = test[training_columns]
y = test[['Survived']]


depth = 2
# Create Decision Tree classifer object
clf1 = DecisionTreeClassifier(max_depth = depth, criterion='gini')
clf2 = DecisionTreeClassifier(max_depth = depth, criterion='gini')
clf3 = DecisionTreeClassifier(max_depth = depth, criterion='gini')


# Train Decision Tree Classifer
clf1 = clf1.fit(X1, y1)
clf2 = clf2.fit(X2, y2)
clf3 = clf3.fit(X3, y3)

# Create Predictions
y['Prediction1'] = clf1.predict(X)
y['Prediction2'] = clf2.predict(X)
y['Prediction3'] = clf3.predict(X)

y['final_prediction'] = y['Prediction1'] + y['Prediction2'] + y['Prediction3']


y['final_prediction'].value_counts()

0    117
3     32
2     30
Name: final_prediction, dtype: int64

### Treffe Vorhersagen mit den Weak Learners basierend auf einer Mehrheitswahl (Majority Vote)

In [ ]:
# Create Predictions based on the predictions of the three weak learners
X = test[training_columns]
y = test[['Survived']]

## TODO
## Tipp: Put the final result into a column named 'Prediction_Final' so that the performance metric is calculated correctly


# Calculate Performance Metric
y['match'] = y['Survived'] == y['Prediction_Final']
performance =  sum(y['match']) / len(y)
print('Performance: {:.1%}'.format(performance))

### Suche Hyperparameter

Es können mehrere Parameter gesucht werden (Anzahl Weak learners, Grösse der Trainingsdaten, Maximale Tiefe des Baums, Voting-Regel, ...) in diesem Fall suchen wir die optimale Grösse der Trainingsdaten mit drei weak learners.

In [ ]:
## TODO